<a href="https://colab.research.google.com/github/SanghaviHarshPankajkumar/shipping_label_project/blob/main/notebooks/Shipping_label_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:

import string

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

def remove_punctuation_and_stopwords(sent):
    # Remove punctuation
    sent = sent.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the sentence
    tokens = nltk.word_tokenize(sent)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Join the tokens back into a sentence
    sent = ' '.join(tokens)

    return sent

# Example usage
sent = "This is a sentence. It has punctuation and stop words."
sent = remove_punctuation_and_stopwords(sent)
print(sent)  # Output: "sentence punctuation stop words"


In [ ]:
!pip install datasets evaluate seqeval

In [ ]:
! pip install -U accelerate
! pip install -U transformers

### Login to huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### import dataset from hugging face

In [ ]:
from datasets import load_dataset

shipping_labels = load_dataset("harsh13333/shipping_label_ner", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

### explore the data

In [ ]:
shipping_labels

In [ ]:
shipping_labels['train']

In [ ]:
shipping_labels['train'].to_pandas()

In [ ]:
### show the labels list
label_list = shipping_labels["train"].features[f"ner_tags"].feature.names
label_list

### Model Creation
- used models
1.  xlm-roberta-large
2. distilbert-base-cased
3. bert-base-uncased
4. bert-base-cased

In [ ]:

from transformers import AutoTokenizer

tokenizer_roberta = AutoTokenizer.from_pretrained("xlm-roberta-large")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:


from transformers import AutoTokenizer

tokenizer_bert_uncased = AutoTokenizer.from_pretrained("bert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:

from transformers import AutoTokenizer
tokenizer_bert = AutoTokenizer.from_pretrained("bert-base-cased")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer_distilbert = AutoTokenizer.from_pretrained("distilbert-base-cased")


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels(examples):
    """Change the tokenizer for specific model"""
    tokenized_inputs = tokenizer_bert_uncased(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_bert_uncased = shipping_labels.map(tokenize_and_align_labels, batched=True)

NameError: name 'shipping_labels' is not defined

In [ ]:
from transformers import DataCollatorForTokenClassification
#add tokenizer
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer_bert_uncased,)

### Train the model

In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
import numpy as np

example = shipping_labels["train"][0]['ner_tags']
labels = [label_list[i] for i in example]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = {
 0:"O",
 1:"B-ORG",
 2:"I-ORG",
 3:"B-GCNUMBER",
 4:"I-GCNUMBER",
 5:"B-BGNUMBER",
 6:"I-BGNUMBER",
 7:"B-NAME",
 8:"I-NAME",
 9:"B-LOCATION",
 10:"I-LOCATION",
 11:"B-COUNTRY",
 12:"I-COUNTRY",
}
label2id = {
   "O" :0 ,
  "B-ORG" :1 ,
  "I-ORG" :2 ,
  "B-GCNUMBER" :3 ,
  "I-GCNUMBER" :4 ,
  "B-BGNUMBER" :5 ,
  "I-BGNUMBER" :6 ,
  "B-NAME" :7 ,
  "I-NAME" :8 ,
  "B-LOCATION" :9 ,
   "I-LOCATION":10 ,
   "B-COUNTRY":11 ,
   "I-COUNTRY":12 ,
}

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=13,id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="ner_bert_uncased_model",
    save_total_limit=1,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=2,
    num_train_epochs=30,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
   model,
   training_args,
    train_dataset=tokenized_bert_uncased["train"],
    eval_dataset=tokenized_bert_uncased["validation"],
    tokenizer=tokenizer_bert_uncased,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.982627,0.000000,0.000000,0.000000,0.429379
2,No log,1.752454,0.307692,0.053333,0.090909,0.491525
3,No log,1.570154,0.384615,0.133333,0.198020,0.514124
4,No log,1.439291,0.366667,0.146667,0.209524,0.525424
5,No log,1.316981,0.393939,0.173333,0.240741,0.553672
6,No log,1.204373,0.583333,0.280000,0.378378,0.604520
7,No log,1.084689,0.596154,0.413333,0.488189,0.661017
8,No log,0.979839,0.650000,0.520000,0.577778,0.711864
9,No log,0.889085,0.685714,0.640000,0.662069,0.757062
10,No log,0.804649,0.718310,0.680000,0.698630,0.774011


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are i

TrainOutput(global_step=210, training_loss=0.47932579403831843, metrics={'train_runtime': 508.6205, 'train_samples_per_second': 2.949, 'train_steps_per_second': 0.413, 'total_flos': 52770861370944.0, 'train_loss': 0.47932579403831843, 'epoch': 30.0})

#### push to hub the final model

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1711602729.7cf60fa1c92f.682.0:   0%|          | 0.00/19.4k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/harsh13333/ner_bert_model/commit/c9b064cf83491ebaf0871839b906851c94c26a6b', commit_message='End of training', commit_description='', oid='c9b064cf83491ebaf0871839b906851c94c26a6b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

# Load the pipeline
nlp = pipeline("ner", model="harsh13333/ner_bert_model",aggregation_strategy="simple")


# Pass a text to the pipeline
text = "LEX2 43Lbs 03/14 OMI6 Jullen Cohen GC11909 A654SW75THAVE 33155FL UnitedStates T8A305725101973 DMI6 CYCLE BHM1 MGES MGES B040 MIA5 DMI6"

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [ ]:
nlp(text)

[{'entity': 'B-NAME',
  'score': 0.9495337,
  'index': 13,
  'word': 'Jul',
  'start': 22,
  'end': 25},
 {'entity': 'I-NAME',
  'score': 0.9651188,
  'index': 15,
  'word': 'Cohen',
  'start': 29,
  'end': 34},
 {'entity': 'B-GCNUMBER',
  'score': 0.96900594,
  'index': 16,
  'word': 'G',
  'start': 35,
  'end': 36},
 {'entity': 'B-LOCATION',
  'score': 0.7842003,
  'index': 30,
  'word': '33',
  'start': 57,
  'end': 59},
 {'entity': 'B-COUNTRY',
  'score': 0.7859303,
  'index': 34,
  'word': 'United',
  'start': 65,
  'end': 71},
 {'entity': 'B-ORG',
  'score': 0.5347484,
  'index': 49,
  'word': 'D',
  'start': 94,
  'end': 95},
 {'entity': 'B-ORG',
  'score': 0.4627055,
  'index': 70,
  'word': 'D',
  'start': 130,
  'end': 131}]

### flair

In [ ]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 57.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB

In [ ]:
!pip install torch

In [ ]:
# directory where the data resides
from flair.datasets import ColumnCorpus
from flair.data import Corpus
data_folder = '/content'
# initializing the corpus
columns = {0 : 'text', 1 : 'ner'}
corpus:Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'val.txt')

2024-04-02 12:38:16,830 Reading data from /content
2024-04-02 12:38:16,832 Train: /content/train.txt
2024-04-02 12:38:16,833 Dev: /content/val.txt
2024-04-02 12:38:16,834 Test: /content/test.txt


In [ ]:
# prompt: hstack merge 2 numpy array

import numpy as np

# Create two numpy arrays
arr1 = np.array([1, 2, 3])
arr2 = np.array([4, 5, 6])

# Concatenate the arrays horizontally (side-by-side)
arr_hstack = np.hstack((arr1, arr2))

# Print the resulting array
print(arr_hstack)


[1 2 3 4 5 6]


In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from flair.embeddings import TransformerWordEmbeddings

# embeddings = TransformerWordEmbeddings(
#     model='distilbert-base-cased',
#     layers="-1",
#     subtoken_pooling="first",
#     fine_tune=True,
#     use_context=True,
# )

# 1. get the corpus
columns = {0 : 'text', 1 : 'ner'}
corpus:Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'val.txt')

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type = 'ner')
print(tag_dictionary)
# 4. initialize each embedding we use
embedding_types = [
    WordEmbeddings('en-glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

# embedding stack consists of Flair and GloVe embeddings
embeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type=tag_type)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer = ModelTrainer(tagger, corpus)

# 7. run training
trainer.train('/content/drive/MyDrive/resources/taggers/ner-english',
              train_with_dev=True,
              max_epochs=150)


2024-04-02 12:38:21,457 Reading data from /content
2024-04-02 12:38:21,460 Train: /content/train.txt
2024-04-02 12:38:21,463 Dev: /content/val.txt
2024-04-02 12:38:21,468 Test: /content/test.txt
2024-04-02 12:38:21,549 Computing label dictionary. Progress:


0it [00:00, ?it/s]
45it [00:00, 15121.27it/s]

2024-04-02 12:38:21,619 Dictionary created for label 'ner' with 2 values: GCNUM (seen 43 times), TRACK-ID (seen 42 times)
Dictionary with 2 tags: GCNUM, TRACK-ID


2024-04-02 12:38:21,903 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpnvmwlbpa


100%|██████████| 153M/153M [00:04<00:00, 38.2MB/s]

...

2024-04-02 12:38:42,596 copying /tmp/tmphmv3s30g to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2024-04-02 12:38:42,755 removing temp file /tmp/tmphmv3s30g
2024-04-02 12:38:43,154 SequenceTagger predicts: Dictionary with 9 tags: O, S-GCNUM, B-GCNUM, E-GCNUM, I-GCNUM, S-TRACK-ID, B-TRACK-ID, E-TRACK-ID, I-TRACK-ID
2024-04-02 12:38:43,551 ----------------------------------------------------------------------------------------------------
2024-04-02 12:38:43,553 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'en-glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDr

100%|██████████| 1/1 [00:00<00:00,  3.71it/s]

2024-04-02 12:39:09,914 
Results:
- F-score (micro) 0.9091
- F-score (macro) 0.9
- Accuracy 0.8333

By class:
              precision    recall  f1-score   support

       GCNUM     1.0000    1.0000    1.0000         3
    TRACK-ID     1.0000    0.6667    0.8000         3

   micro avg     1.0000    0.8333    0.9091         6
   macro avg     1.0000    0.8333    0.9000         6
weighted avg     1.0000    0.8333    0.9000         6

2024-04-02 12:39:09,916 ----------------------------------------------------------------------------------------------------


{'test_score': 0.9090909090909091}

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
model = SequenceTagger.load('/content/resources/taggers/ner-english/final-model.pt')
# create example sentence
sentence = Sentence('DOOMDAY WHITESTOWN INADS dipak manavr dipak manvar GC14024  INDIA FL55 BG2303140267275 GROUND OnTrac')
# predict the tags
model.predict(sentence)
print(sentence.to_tagged_string())

2024-04-02 06:50:57,549 SequenceTagger predicts: Dictionary with 11 tags: O, S-GCNUM, B-GCNUM, E-GCNUM, I-GCNUM, S-TRACK-ID, B-TRACK-ID, E-TRACK-ID, I-TRACK-ID, <START>, <STOP>
Sentence[13]: "DOOMDAY WHITESTOWN INADS dipak manavr dipak manvar GC14024  INDIA FL55 BG2303140267275 GROUND OnTrac" → ["GC14024"/GCNUM, "BG2303140267275"/TRACK-ID]


In [ ]:
import torch
torch.save(model.state_dict(), '/content/drive/MyDrive/model.bin')

In [ ]:
corpus.train[0].to_tagged_string('ner')

'Sentence[10]: "ROADGET LINTHA SMITH GC13124 MIAMI FL33155 BG2303140267275 GROUND OnTrac LINTA" → ["ROADGET"/ORG, "LINTHA SMITH"/NAME, "GC13124"/GCNUMBER, "MIAMI FL33155"/LOCATION, "BG2303140267275"/BGNUMBER, "LINTA"/NAME]'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip freeze

absl-py==1.4.0
accelerate==0.28.0
aiohttp==3.9.3
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.0.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
boto3==1.34.72
botocore==1.34.72
bpemb==0.3.5
bqplot==0.12.43
branca==0.7.1
build==1.1.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.1
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
conllu==4.5.3
cons==0.4.6
contextlib2==21.6.0

In [ ]:
!pip freeze

absl-py==1.4.0
accelerate==0.28.0
aiohttp==3.9.3
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.0.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
boto3==1.34.75
botocore==1.34.75
bpemb==0.3.5
bqplot==0.12.43
branca==0.7.1
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.2
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
conllu==4.5.3
cons==0.4.6
contextlib2==21.6.0

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install pandas_profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 13.3 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=51cdf362db3a9c45d6f23d79c1a47acb04f8eb3c869b8cb9dc6bb4efa92eae5e
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.13.1
    Uninstalling seaborn-0.13.1:
      Successfully uninstalled seaborn-0.13.1


In [ ]:
# prompt: pandas profiler

import pandas as pd
# Load your dataset into a pandas DataFrame
df = pd.read_csv('details.csv')




In [ ]:

import pandas as pd

In [ ]:
pd.DataFrame(
   {'s1':[1,2,3,4],'s2':[3,4,5,6]}
)

,s1,s2
0,1,3
1,2,4
2,3,5
3,4,6
